In [1]:
# Import modules
import pandas as pd
import numpy as np

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [2]:
# Save path to data set in a variable
data_file1 = "Resources/clean_hawaii_measurements.csv"
data_file2 = "Resources/hawaii_stations.csv"

# Load the cleaned csv file into a pandas dataframe
measurement_df = pd.read_csv(data_file1)
print (measurement_df.count())
print (measurement_df.dtypes)
measurement_df.head()

station    18103
date       18103
prcp       18103
tobs       18103
dtype: int64
station     object
date        object
prcp       float64
tobs         int64
dtype: object


,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [3]:
station_df = pd.read_csv(data_file2)
print (station_df.count())
print (station_df.dtypes)
station_df.head()

station      9
name         9
latitude     9
longitude    9
elevation    9
dtype: int64
station       object
name          object
latitude     float64
longitude    float64
elevation    float64
dtype: object


,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [4]:
# Create engine using the `hawaii.sqlite` database file
engine = create_engine("sqlite:///hawaii.sqlite")

# Create a connection to the engine called `conn`
conn = engine.connect()

In [5]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
# Create Measurements and Stations Classes

Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String(128))
    date = Column(String(30))
    prcp = Column(Float)
    tobs = Column(Float)

class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(128))
    name = Column(String(128))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [6]:
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [7]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
measurement_data = measurement_df.to_dict(orient='records')

station_data = station_df.to_dict(orient='records')

In [8]:
# Data is just a list of dictionaries that represent each row of data
print(measurement_data[:5])

print(station_data[:5])

[{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}, {'station': 'USC00519397', 'date': '2010-01-02', 'prcp': 0.0, 'tobs': 63}, {'station': 'USC00519397', 'date': '2010-01-03', 'prcp': 0.0, 'tobs': 74}, {'station': 'USC00519397', 'date': '2010-01-04', 'prcp': 0.0, 'tobs': 76}, {'station': 'USC00519397', 'date': '2010-01-07', 'prcp': 0.06, 'tobs': 70}]
[{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}, {'station': 'USC00513117', 'name': 'KANEOHE 838.1, HI US', 'latitude': 21.4234, 'longitude': -157.8015, 'elevation': 14.6}, {'station': 'USC00514830', 'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 'latitude': 21.5213, 'longitude': -157.8374, 'elevation': 7.0}, {'station': 'USC00517948', 'name': 'PEARL CITY, HI US', 'latitude': 21.3934, 'longitude': -157.9751, 'elevation': 11.9}, {'station': 'USC00518838', 'name': 'UPPER WAHIAWA 874.3, HI US', 'latitude': 21.4992, 'longitude': -158.0111, 'e

In [9]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [10]:
# Save the reference to the `measurements` table as a variable called `table`
measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)

station_table = sqlalchemy.Table('station', metadata, autoload=True)

In [11]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(measurement_table.delete())

conn.execute(station_table.delete())

In [12]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(measurement_table.insert(), measurement_data)

conn.execute(station_table.insert(), station_data)

In [13]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [14]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]